### Unseen Dataset Testing

In [ ]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [ ]:
!pip install -e '.[dev]'
!pip install rouge_score
!pip install sentence_transformers

Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=de59f0e11e502ec964f066452d183add2c44c0367f104e266c50e46fdb2b437d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using c

In [ ]:
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, AdamW
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
model_name="/content/drive/MyDrive/modelv3"
#/content/drive/MyDrive/modelv2
#/content/drive/MyDrive/Gov_Model
tokenizer = MBart50TokenizerFast.from_pretrained('facebook/mbart-large-50')
model = MBartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Dataset/test_data.xlsx')
data = data[['Q','A']]
data.dropna(inplace=True)

In [ ]:
data.head()

,Q,A
0,മരണം ഉടനടി രജിസ്റ്റർ ചെയ്തില്ലെങ്കിൽ എനിക്ക് മ...,"അതെ, മരണം ഉടനടി രജിസ്റ്റർ ചെയ്തില്ലെങ്കിലും നി..."
1,ഒരു എൻആർഐക്ക് വരുമാന സർട്ടിഫിക്കറ്റിന് അപേക്ഷി...,"ഇല്ല, പ്രത്യേക സംസ്ഥാന/യൂണിയൻ ടെറിട്ടറിയിലെ താ..."
2,ഒരു വരുമാന സർട്ടിഫിക്കറ്റ് എത്ര കാലത്തേക്ക് സാ...,സർട്ടിഫിക്കറ്റിൽ വ്യക്തമാക്കിയ സാമ്പത്തിക വർഷത...
3,ഒറിജിനൽ നഷ്ടപ്പെട്ടാൽ എനിക്ക് എങ്ങനെ ഡ്യൂപ്ലിക...,നിങ്ങളുടെ വരുമാന സർട്ടിഫിക്കറ്റ് നഷ്ടപ്പെട്ടാൽ...
4,ആരാണ് നേറ്റിവിറ്റി സർട്ടിഫിക്കറ്റ് നൽകുന്നത്?,നേറ്റിവിറ്റി സർട്ടിഫിക്കറ്റ് നൽകുന്ന അധികാരം ഓ...


In [ ]:
questions=list(data['Q'])
answers=list(data['A'])

##### Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (fi

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
def calculate_rouge(reference, hypothesis):
    score = scorer.score(reference, hypothesis)
    return score

In [ ]:
model.eval()
max_length=128
correct_predictions = 0
predicted_answer=[]
total_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
for i in range(len(questions)):
    input_text = questions[i]
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=max_length, padding='max_length', truncation=True).input_ids.to(device)
    output_ids = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
    predicted_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predicted_answer.append(predicted_text)
    print(f"Question: {input_text}")
    print(f"Predicted Answer: {predicted_answer[i]}")
    score = calculate_rouge(answers[i].strip(),predicted_text.strip())
    total_scores['rouge1'] += score['rouge1'].fmeasure
    total_scores['rouge2'] += score['rouge2'].fmeasure
    total_scores['rougeL'] += score['rougeL'].fmeasure
    if predicted_text == answers[i].strip():
        correct_predictions += 1

Question: മരണം ഉടനടി രജിസ്റ്റർ ചെയ്തില്ലെങ്കിൽ എനിക്ക് മരണ സർട്ടിഫിക്കറ്റ് ലഭിക്കുമോ?
Predicted Answer: മരണ സർട്ടിഫിക്കറ്റ് ഒന്നുകിൽ ഒരു വ്യക്തി എപ്പോൾ മരിച്ചുവെന്നു വ്യക്തമാക്കുന്ന ഒരു മെഡിക്കൽ പ്രാക്ടീഷണർ നൽകുന്ന നിയമപരമായ രേഖയാണ്, അല്ലെങ്കിൽ ഒരു ഔദ്യോഗിക രജിസ്റ്ററിൽ രേഖപ്പെടുത്തിയിരിക്കുന്ന ഒരു വ്യക്തിയുടെ മരണ തീയതിയും സ്ഥലവും കാരണവും പ്രഖ്യാപിക്കുന്ന ഒരു സർക്കാർ സിവിൽ രജിസ്ട്രേഷൻ ഓഫീസ് നൽകുന്ന ഒരു രേഖയാണ്. മരണങ്ങളുടെ.
Question: ഒരു എൻആർഐക്ക് വരുമാന സർട്ടിഫിക്കറ്റിന് അപേക്ഷിക്കാമോ?
Predicted Answer: കേരളത്തിലെ വരുമാന സർട്ടിഫിക്കറ്റിന് അപേക്ഷിക്കുന്നതിന് ഇനിപ്പറയുന്ന രേഖകൾ ആവശ്യമാണ്: റേഷൻ കാർഡ്, തിരിച്ചറിയാനുള്ള തെളിവ് വരുമാന തെളിവ്, ഭൂനികുതി, ശമ്പള സർട്ടിഫിക്കറ്റ്, വരുമാന സർട്ടിഫിക്കറ്റ് ലഭിക്കുന്നതിനുള്ള കാരണങ്ങൾ
Question: ഒരു വരുമാന സർട്ടിഫിക്കറ്റ് എത്ര കാലത്തേക്ക് സാധുവാണ്?
Predicted Answer: കേരളത്തിലെ വരുമാന സർട്ടിഫിക്കറ്റ് ലഭിക്കുന്നതിനുള്ള ചില പ്രധാന കാരണങ്ങൾ ഇവയാണ്: വസ്തു വാങ്ങുന്ന സമയത്ത് നികുതി ഇളവ് ലഭിക്കുന്നതിന് വരുമാന സർട്ടിഫിക്കറ്റ് ഹാജരാക്കേണ്ടതുണ്ട്, ഒരു വിദ്യാഭ്യാസ സ

##### BLEU Score

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

bleu_scores = []

for i in range(len(answers)):
    predict = predicted_answer[i].strip()
    actual = answers[i].strip()

    # Tokenize predicted and actual answers using tokenizer
    tokenized_predict = tokenizer(predict, return_tensors="pt", max_length=128, truncation=True, padding=True)["input_ids"]
    tokenized_actual = tokenizer(actual, return_tensors="pt", max_length=128, truncation=True, padding=True)["input_ids"]

    # Decode token IDs to strings
    tokenized_predict = tokenizer.decode(tokenized_predict[0], skip_special_tokens=True)
    tokenized_actual = tokenizer.decode(tokenized_actual[0], skip_special_tokens=True)

    # Calculate BLEU score for current pair of answers
    bleu = corpus_bleu([[tokenized_actual]], [tokenized_predict])
    bleu_scores.append(bleu)
    print("BLEU score for instance", i+1, ":", bleu)

avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average Bleu Score:", avg_bleu_score)


BLEU score for instance 1 : 0.312871604047126
BLEU score for instance 2 : 0.3765014708814825
BLEU score for instance 3 : 0.3647826601790706
BLEU score for instance 4 : 0.16739665698902317
BLEU score for instance 5 : 0.40009003951523275
BLEU score for instance 6 : 0.177091929164624
BLEU score for instance 7 : 0.018534593210730865
BLEU score for instance 8 : 0.09525305581378937
BLEU score for instance 9 : 0.37444482122898226
BLEU score for instance 10 : 0.33861129098918463
BLEU score for instance 11 : 0.26283224476507466
BLEU score for instance 12 : 0.13375495425098427
BLEU score for instance 13 : 0.3636981865466051
BLEU score for instance 14 : 0.0024167574291083785
BLEU score for instance 15 : 0.12454993165211888
BLEU score for instance 16 : 0.24644362246757068
BLEU score for instance 17 : 0.45845769034185696
BLEU score for instance 18 : 0.5092742622246483
BLEU score for instance 19 : 0.37306232823788354
BLEU score for instance 20 : 0.31026612306718293
Average Bleu Score: 0.270516711150

##### Rouge Score

In [ ]:
num_instances = len(questions)
avg_rouge1 = total_scores['rouge1'] / num_instances
avg_rouge2 = total_scores['rouge2'] / num_instances
avg_rougeL = total_scores['rougeL'] / num_instances

print("Average ROUGE-1 F1 Score:", avg_rouge1)
print("Average ROUGE-2 F1 Score:", avg_rouge2)
print("Average ROUGE-L F1 Score:", avg_rougeL)

Average ROUGE-1 F1 Score: 0.022222222222222223
Average ROUGE-2 F1 Score: 0.014285714285714287
Average ROUGE-L F1 Score: 0.022222222222222223


##### Cosine Simiarity

In [ ]:
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
model_accuracy = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import numpy as np
actual_embeddings = model_accuracy.encode(answers, convert_to_tensor=True)
predicted_embeddings = model_accuracy.encode(predicted_answer, convert_to_tensor=True)
cos_sim = util.pytorch_cos_sim(actual_embeddings, predicted_embeddings)
cos_similarities = cos_sim.cpu().numpy()
average_cos_similarity = np.mean(np.diagonal(cos_similarities))


In [ ]:
print("Cosine Sim:",average_cos_similarity)

Cosine Sim: 0.78768575


In [ ]:
for i in range(0,len(questions)):
    print(cos_similarities[i][i])

0.9490329
0.93626404
0.90777254
0.86210096
0.88233924
0.93831444
0.8460119
0.77645487
0.84589064
0.9518374
0.840885
0.844233
0.7131567
0.12951712
0.90041494
0.28564394
0.9327046
0.9766458
0.47355136
0.760943


In [ ]:
for i in range(0,len(predicted_answer)):
  print(len(predicted_answer[i]),"    ",len(answers[i]))

321      628
218      314
560      285
394      157
392      347
167      280
82      374
118      303
386      294
167      199
293      257
300      131
293      242
58      357
185      846
168      234
223      236
405      946
372      257
382      290


### Seen Dataset Testing

##### Data Pre-Process

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Dataset/all cerificates(updated).xlsx')
data = data[['Q','A']]
data.dropna(inplace=True)

<ipython-input-10-61e36dde3bd8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 109
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       100 non-null    object
 1   A       100 non-null    object
dtypes: object(2)
memory usage: 2.3+ KB


In [ ]:
data.head()

,Q,A
0,എന്താണ് വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റ്?,"ഒരു വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റ്, സാധാരണയാ..."
1,കേരളത്തിൽ ഒരു വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റി...,റീജിയണൽ ട്രാൻസ്‌പോർട്ട് ഓഫീസിൽ (ആർടിഒ) അല്ലെങ്...
2,വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റിന് അപേക്ഷിക്കാ...,ആവശ്യമായ രേഖകളിൽ സാധാരണയായി ഉൾപ്പെടുന്നു:\n\nഅ...
3,വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റ് ലഭിക്കാൻ എത്ര...,RTO-യിലെ ജോലിഭാരവും ഡോക്യുമെൻ്റേഷൻ്റെ സമ്പൂർണ്...
4,വാഹന രജിസ്ട്രേഷൻ സർട്ടിഫിക്കറ്റ് ലഭിക്കുന്നതിന...,"അതെ, രജിസ്ട്രേഷൻ ചാർജുകൾ, റോഡ് ടാക്സ്, മറ്റ് ബ..."


In [ ]:
questions = list(data['Q'])
answers = list(data['A'])

In [ ]:
from sklearn.model_selection import train_test_split
train_questions,test_questions,train_answers,test_answers = train_test_split(questions,answers,test_size=0.2,random_state=42)

##### Model Prediction

###### Additional training done to increase model performance 20 epochs

In [ ]:
max_length=128
def tokenize_data(value):
    input_ids = []
    attention_masks = []
    for item in value:
        tensor_values = tokenizer(item, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt")
        input_ids.append(tensor_values['input_ids'])
        attention_masks.append(tensor_values['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

In [ ]:
train_question_ids, train_question_attention = tokenize_data(questions)
train_answer_ids, train_answer_attention = tokenize_data(answers)

In [ ]:
batch_size = 8
from torch.utils.data import Dataset, DataLoader,TensorDataset
train_dataset = TensorDataset(train_question_ids, train_question_attention, train_answer_ids, train_answer_attention)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 10
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        decoder_attention_mask = batch[3].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_loss = total_loss / len(train_loader)
    print(f'Train Loss: {avg_loss:.4f}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
Train Loss: 0.0256
Epoch 2/10
Train Loss: 0.0265
Epoch 3/10
Train Loss: 0.0240
Epoch 4/10
Train Loss: 0.0244
Epoch 5/10
Train Loss: 0.0214
Epoch 6/10
Train Loss: 0.0211
Epoch 7/10
Train Loss: 0.0190
Epoch 8/10
Train Loss: 0.0185
Epoch 9/10
Train Loss: 0.0170
Epoch 10/10
Train Loss: 0.0156


In [ ]:
model.save_pretrained('/content/drive/MyDrive/modelv4')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}



###### Model Evaluation

In [ ]:
model.eval()
max_length=128
correct_predictions = 0
predicted_answer=[]
total_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
for i in range(0,len(test_questions)):
  input_text = test_questions[i]
  input_ids = tokenizer(input_text,return_tensors="pt",truncation=True,padding='max_length',max_length=max_length)['input_ids'].to(device)
  outputs = model.generate(input_ids,max_length=max_length,num_beams=5,early_stopping=True)
  predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predicted_answer.append(predicted_text)
  print(f"Question: {input_text}")
  print(f"Predicted Answer: {predicted_text}")
  score = calculate_rouge(test_answers[i].strip(),predicted_text.strip())
  total_scores['rouge1'] += score['rouge1'].fmeasure
  total_scores['rouge2'] += score['rouge2'].fmeasure
  total_scores['rougeL'] += score['rougeL'].fmeasure
  if predicted_text == test_answers[i].strip():
        correct_predictions += 1


Question: ഒരു OBC സർട്ടിഫിക്കറ്റിനുള്ള സാധുതയുടെ കാലാവധി എത്രയാണ്?
Predicted Answer: കേന്ദ്ര സർക്കാർ നൽകുന്ന ഒബിസി (എൻസിഎൽ) സർട്ടിഫിക്കറ്റിൻ്റെ സാധുത ഇഷ്യൂ ചെയ്ത തീയതി മുതൽ ഒരു വർഷമാണ്. എന്നിരുന്നാലും, സംസ്ഥാന സർക്കാർ നൽകുന്ന സർട്ടിഫിക്കറ്റുകൾക്ക് മൂന്ന് വർഷത്തെ കാലാവധിയുണ്ട്. പ്രാരംഭ സാധുത കാലാവധി അവസാനിച്ചതിന് ശേഷം സർട്ടിഫിക്കറ്റ് പുതുക്കുന്നതിന് അപേക്ഷിക്കാവുന്നതാണ്.
Question: ഡൊമിസൈൽ സർട്ടിഫിക്കറ്റ് ലഭിക്കുന്നതിനുള്ള കാരണങ്ങൾ?
Predicted Answer: ഡൊമിസൈൽ സർട്ടിഫിക്കറ്റ് ഉപയോഗിച്ച് വിദ്യാർത്ഥികൾക്ക് ഒരു വിദ്യാഭ്യാസ സ്ഥാപനത്തിൽ പ്രവേശനം നേടാം ഒരു ക്ലെയിം തെളിയിക്കാൻ, ഒരു റേഷൻ കാർഡിന്, അത് ഒരു പ്രധാന രേഖയാണ് വിദ്യാഭ്യാസത്തിൽ ക്വാട്ട ലഭിക്കാൻ ഡൊമിസൈൽ സർട്ടിഫിക്കറ്റും ഉപയോഗിക്കുന്നു താമസക്കാർക്ക് മുൻഗണന നൽകുന്ന സർക്കാർ ജോലികൾക്ക് അപേക്ഷിക്കാൻ ഉപയോക്താവിന് ഡൊമിസൈൽ സർട്ടിഫിക്കറ്റ് ഉപയോഗിക്കാം സ്കോളർഷിപ്പ് സ്കീമുകൾക്ക് അപേക്ഷിക്കാൻ വിദ്യാർത്ഥികൾക്ക് ഈ സർട്ടിഫിക്കറ്റ് ഉപയോഗിക്കാം
Question: എൻ്റെ നേറ്റിവിറ്റി സർട്ടിഫിക്കറ്റിൽ ഒരു പിശകോ പൊരുത്തക്കേടോ ഉണ്ടെങ്കിൽ ഞാൻ എന്തുചെയ്യണം?
Predicted Answe

In [ ]:
for i in range(0,len(predicted_answer)):
  print(len(predicted_answer[i]),"    ",len(test_answers[i]))

287      290
432      432
496      789
417      480
442      546
378      640
468      468
485      878
107      107
211      211
430      960
414      414
407      465
556      753
320      321
208      209
404      772
295      295
369      450
218      220


##### BLEU Score

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

bleu_scores = []

for i in range(len(test_answers)):
    predict = predicted_answer[i].strip()
    actual = test_answers[i].strip()

    # Tokenize predicted and actual answers using tokenizer
    tokenized_predict = tokenizer(predict, return_tensors="pt", max_length=128, truncation=True, padding=True)["input_ids"]
    tokenized_actual = tokenizer(actual, return_tensors="pt", max_length=128, truncation=True, padding=True)["input_ids"]

    # Decode token IDs to strings
    tokenized_predict = tokenizer.decode(tokenized_predict[0], skip_special_tokens=True)
    tokenized_actual = tokenizer.decode(tokenized_actual[0], skip_special_tokens=True)

    # Calculate BLEU score for current pair of answers
    bleu = corpus_bleu([[tokenized_actual]], [tokenized_predict])
    bleu_scores.append(bleu)
    print("BLEU score for instance", i+1, ":", bleu)

# Calculate average BLEU score
print(len(bleu_scores))
print(len(test_questions))
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average Bleu Score:", avg_bleu_score)


BLEU score for instance 1 : 0.9809128687657304
BLEU score for instance 2 : 1.0
BLEU score for instance 3 : 0.9959758605722213
BLEU score for instance 4 : 0.9928315720259139
BLEU score for instance 5 : 0.9909910521988199
BLEU score for instance 6 : 0.9920949029899864
BLEU score for instance 7 : 1.0
BLEU score for instance 8 : 0.9917864940042292
BLEU score for instance 9 : 1.0
BLEU score for instance 10 : 1.0
BLEU score for instance 11 : 0.9953596371164251
BLEU score for instance 12 : 1.0
BLEU score for instance 13 : 0.9902201272343365
BLEU score for instance 14 : 0.9892666510083027
BLEU score for instance 15 : 0.992166424344932
BLEU score for instance 16 : 0.9879304656603336
BLEU score for instance 17 : 0.9975278133743868
BLEU score for instance 18 : 0.9914600338987654
BLEU score for instance 19 : 0.9892184338482631
BLEU score for instance 20 : 0.9884863661233536
20
20
Average Bleu Score: 0.9933114351582999


##### Rouge Score

In [ ]:
num_instances = len(test_questions)
avg_rouge1 = total_scores['rouge1'] / num_instances
avg_rouge2 = total_scores['rouge2'] / num_instances
avg_rougeL = total_scores['rougeL'] / num_instances

print("Average ROUGE-1 F1 Score:", avg_rouge1)
print("Average ROUGE-2 F1 Score:", avg_rouge2)
print("Average ROUGE-L F1 Score:", avg_rougeL)

Average ROUGE-1 F1 Score: 0.34285714285714286
Average ROUGE-2 F1 Score: 0.29166666666666663
Average ROUGE-L F1 Score: 0.34285714285714286


##### Cosine Similarity

In [ ]:
import numpy as np
actual_embeddings = model_accuracy.encode(test_answers, convert_to_tensor=True)
predicted_embeddings = model_accuracy.encode(predicted_answer, convert_to_tensor=True)
cos_sim = util.pytorch_cos_sim(actual_embeddings, predicted_embeddings)
cos_similarities = cos_sim.cpu().numpy()
average_cos_similarity = np.mean(np.diagonal(cos_similarities))
print("Cosine Sim:",average_cos_similarity)

Cosine Sim: 0.9974184


In [ ]:
for i in range(0,len(test_questions)):
    print(cos_similarities[i][i])

0.99959314
0.9999999
0.9994158
0.9967989
0.9990223
0.9993265
0.99999994
0.9982358
0.99999994
0.9999999
0.9984467
1.0
0.99946
0.9977872
0.9974711
0.9982176
0.9975575
0.9695938
0.9988965
0.9985485
